<a href="https://colab.research.google.com/github/cbologa/public_databases/blob/main/python/ChEMBL_get_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title
%load_ext google.colab.data_table

Install and import required packages

In [ ]:
#@title
!pip install -q chembl-downloader
import chembl_downloader

Download last version of ChEMBL

In [ ]:
#@title
path = chembl_downloader.download_extract_sqlite(version='31')

Define SQL query

In [ ]:
#@title
sql = """
select
  target_dictionary.*,
  assays.*,
  activities.*,
  compound_structures.standard_inchi_key,
  compound_structures.canonical_smiles,
  molecule_dictionary.*,
  docs.*
from
  target_dictionary
  left join assays on target_dictionary.tid=assays.tid
  left join docs on assays.doc_id=docs.doc_id
  left join activities on assays.assay_id=activities.assay_id
  left join molecule_dictionary on activities.molregno=molecule_dictionary.molregno
  left join compound_structures on molecule_dictionary.molregno=compound_structures.molregno
where 
  target_dictionary.organism like '%equine encephalitis virus'
group by
  target_dictionary.tid,
  assays.assay_id,
  activities.activity_id,
  compound_structures.standard_inchi_key,
  compound_structures.canonical_smiles,
  molecule_dictionary.molregno,
  docs.doc_id
order by
  docs.year,
  docs.doi
"""

Query database and save to file

In [ ]:
#@title
df = chembl_downloader.query(sql)

#Save to file
df.to_csv("eev.tsv", sep='\t', index=False)

#Show results
df